## **CHICAGO 2019**

El archivo de chicago 2019 no tiene datos de posición geográfica, simplemente tiene los ids de las estaciones y el nombre la estación. Para poder crear una tabla canónica debe tener la misma información que el resto de tablas. Por lo que se necesita extraer la informacion de latitud y longitud. Para ello, utilizaremos la información del año siguiente, 2020, que sí recoge información geográfica.

# Extracción para 2020

Se quiere extraer el archivo de 2020 para obtener una tabla que relacione los id de las estaciones con la longitud y latitud.

In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import pandas as pd

# Definimos la URL y las columnas necesarias
url = 'https://divvy-tripdata.s3.amazonaws.com/'

# Hacemos la solicitud a la página y usamos BeautifulSoup para obtener el listado de archivos
response = requests.get(url)
soup = BeautifulSoup(response.text, 'xml')

# Extraemos los nombres de archivos y filtramos para obtener solo los de 2020
files = [file.get_text() for file in soup.find_all('Key') if file.get_text().endswith('.zip')]
# Filtrar los archivos que contienen '202001', '202002', ...
files_2020 = [file for file in files if any(f'2020{str(month).zfill(2)}' in file for month in range(1, 13))]


# Transformaciones 2020

In [2]:
# T.1.a) TRANSFORMACIÓN DE FILTRADO: definimos el nombre del archivo CSV final y las columnas requeridas.
output_file = 'Chicago_2020.csv'
required_columns = ['start_station_id', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng'] #se filtran por las columnas requeridas (id y latitud y longitud)

In [3]:
# Verificamos si el archivo de salida ya existe para no sobreescribirlo
if not os.path.exists(output_file):
    # Crear el archivo CSV inicial con la columna adicional 'city'
    pd.DataFrame(columns=required_columns).to_csv(output_file, index=False)

# Iteramos sobre cada archivo mensual de 2020
for file_name in files_2020:
    print(f"Procesando archivo: {file_name}")

    # Descargar el archivo y descomprimir
    file_url = url + file_name
    local_zip = file_name.split('/')[-1]

    with open(local_zip, "wb") as f: #se crea un archivo vacio para meter la información de la URL de cada mes
        f.write(requests.get(file_url).content)

    # Descomprimir el archivo en una carpeta temporal
    with zipfile.ZipFile(local_zip, "r") as zip_ref: #se abre el archivo ZIP para leerlo
        zip_ref.extractall("temp_data")

    # Cargar el CSV descomprimido y filtrar las columnas necesarias
    csv_file = [file for file in os.listdir("temp_data") if file.endswith('.csv')][0]
    df = pd.read_csv(os.path.join("temp_data", csv_file), usecols=required_columns)

    # Guardar los datos en el archivo CSV final (añadir al archivo existente)
    df.to_csv(output_file, mode='a', index=False, header=False)

    # Limpiar archivos temporales
    os.remove(local_zip)
    os.remove(os.path.join("temp_data", csv_file))

print(f"Archivo '{output_file}' actualizado con todos los meses de 2020.")

Procesando archivo: 202004-divvy-tripdata.zip
Procesando archivo: 202005-divvy-tripdata.zip
Procesando archivo: 202006-divvy-tripdata.zip
Procesando archivo: 202007-divvy-tripdata.zip
Procesando archivo: 202008-divvy-tripdata.zip
Procesando archivo: 202009-divvy-tripdata.zip
Procesando archivo: 202010-divvy-tripdata.zip
Procesando archivo: 202011-divvy-tripdata.zip
Procesando archivo: 202012-divvy-tripdata.zip
Archivo 'Chicago_2020.csv' actualizado con todos los meses de 2020.


In [4]:
df = pd.read_csv('Chicago_2020.csv')
df.head()

<ipython-input-4-4aeabe9d1955>:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Chicago_2020.csv')


,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng
0,86,152.0,41.8964,-87.6610,41.9322,-87.6586
1,503,499.0,41.9244,-87.7154,41.9306,-87.7238
2,142,255.0,41.8945,-87.6179,41.8679,-87.6230
3,216,657.0,41.9030,-87.6975,41.8992,-87.6722
4,125,323.0,41.8902,-87.6262,41.9695,-87.6547


Esta tabla recoge la información con respecto al id de las estaciones y su latitud y longitud de los meses de abril a diciembre de 2020. Estos son los primeros datos de posición geográficas que se recogen en divvy bikes.

### Mapeo de id longitud y latitud

El objetivo ahora, es crear una tabla relacional para que a cada id se le asigne una latitud y una longitud.

In [5]:
# Crear un DataFrame con las estaciones de inicio: siempre esta estructura -> id, latitud y longitu
start_stations = df[['start_station_id', 'start_lat', 'start_lng']].rename(
    columns={'start_station_id': 'id_estacion', 'start_lat': 'latitud', 'start_lng': 'longitud'}
)

# Crear un DataFrame con las estaciones de fin: siempre esta estructura -> id, latitud y longitud
end_stations = df[['end_station_id', 'end_lat', 'end_lng']].rename(
    columns={'end_station_id': 'id_estacion', 'end_lat': 'latitud', 'end_lng': 'longitud'}
)

# Unir ambos DataFrames uno debajo del otro y eliminar duplicados
stations = pd.concat([start_stations, end_stations]).drop_duplicates(subset='id_estacion') #se considera ID estacion para identificar los duplicados

stations


,id_estacion,latitud,longitud
0,86,41.896400,-87.661000
1,503,41.924400,-87.715400
2,142,41.894500,-87.617900
3,216,41.903000,-87.697500
4,125,41.890200,-87.626200
...,...,...,...
3056477,20113,41.707342,-87.607690
3058225,649,41.779854,-87.634584
3070810,652,41.766015,-87.611732
3073962,556,41.799659,-87.657195


Esta tabla recoge la situación geográfica de todas las estaciones que había en 2020. Ahora el objetivo es identificiar la posición geográfica para cada id de las estaciones del año 2019.

# Extracción para 2019

In [6]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import pandas as pd

# Definimos la URL y las columnas necesarias
url = 'https://divvy-tripdata.s3.amazonaws.com/'

# Hacemos la solicitud a la página y usamos BeautifulSoup para obtener el listado de archivos
response = requests.get(url)
soup = BeautifulSoup(response.text, 'xml')

# Extraemos los nombres de archivos y filtramos para obtener solo los de 2019
files = [file.get_text() for file in soup.find_all('Key') if file.get_text().endswith('.zip')]
files_2019 = [file for file in files if '2019' in file]

# Transformación

### Transformación 1: filtrado de columnas y transformación 2: creación campo "ciudad"

In [7]:
# T.1.a) TRANSFORMACIÓN DE FILTRADO: definimos el nombre del archivo CSV final y las columnas requeridas.
output_file = 'Chicago_2019.csv'
# Las columnas que necesitas (índices basados en 0). Las columnas tinen distintos nombres pero el mismo orden, se seleccionan números de columnas en vez de nombres de encabezados por facilidad
column_indices = [1, 2, 5, 6, 7, 8, 9]

In [8]:
# Verificamos si el archivo de salida ya existe para no sobreescribirlo
if not os.path.exists(output_file):
    # Crear el archivo CSV inicial con la columna adicional 'city'
    pd.DataFrame(columns=['start_time', 'end_time', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'city']).to_csv(output_file, index=False) #nombre de columnas

# Iteramos sobre cada archivo cuatrimestral de 2019
for file_name in files_2019:
    if '2019' not in file_name:
        continue  # Saltar archivos que no sean del 2019

    print(f"Procesando archivo: {file_name}")

    # Descargar el archivo y descomprimir
    file_url = url + file_name
    local_zip = file_name.split('/')[-1]

    with open(local_zip, "wb") as f:
        f.write(requests.get(file_url).content)

    # Descomprimir el archivo en una carpeta temporal
    with zipfile.ZipFile(local_zip, "r") as zip_ref:
        zip_ref.extractall("temp_data")

    # Cargar el archivo CSV descomprimido y seleccionar columnas por índice
    csv_file = [file for file in os.listdir("temp_data") if file.endswith('.csv')][0]
    df = pd.read_csv(os.path.join("temp_data", csv_file), usecols=column_indices)

    # Renombrar las columnas para unificar el formato
    df.columns = ['start_time', 'end_time', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype']

    # Añadir la columna 'city' con el valor "Chicago"
    df['city'] = 'Chicago'

    # Guardar los datos en el archivo CSV final (añadir al archivo existente)
    df.to_csv(output_file, mode='a', index=False, header=False)

    # Limpiar archivos temporales creados para esta sección
    os.remove(local_zip)
    os.remove(os.path.join("temp_data", csv_file))

print(f"Archivo '{output_file}' actualizado con todos los meses de 2019 y la columna 'city' añadida.")

Procesando archivo: Divvy_Trips_2019_Q1.zip
Procesando archivo: Divvy_Trips_2019_Q2.zip
Procesando archivo: Divvy_Trips_2019_Q3.zip
Procesando archivo: Divvy_Trips_2019_Q4.zip
Archivo 'Chicago_2019.csv' actualizado con todos los meses de 2019 y la columna 'city' añadida.


In [9]:
df = pd.read_csv('Chicago_2019.csv')
df

,start_time,end_time,from_station_id,from_station_name,to_station_id,to_station_name,usertype,city
0,2019-01-01 00:04:37,2019-01-01 00:11:07,199,Wabash Ave & Grand Ave,84,Milwaukee Ave & Grand Ave,Subscriber,Chicago
1,2019-01-01 00:08:13,2019-01-01 00:15:34,44,State St & Randolph St,624,Dearborn St & Van Buren St (*),Subscriber,Chicago
2,2019-01-01 00:13:23,2019-01-01 00:27:12,15,Racine Ave & 18th St,644,Western Ave & Fillmore St (*),Subscriber,Chicago
3,2019-01-01 00:13:45,2019-01-01 00:43:28,123,California Ave & Milwaukee Ave,176,Clark St & Elm St,Subscriber,Chicago
4,2019-01-01 00:14:52,2019-01-01 00:20:56,173,Mies van der Rohe Way & Chicago Ave,35,Streeter Dr & Grand Ave,Subscriber,Chicago
...,...,...,...,...,...,...,...,...
3817999,2019-12-31 23:56:13,2020-01-01 00:15:45,112,Green St & Randolph St,225,Halsted St & Dickens Ave,Subscriber,Chicago
3818000,2019-12-31 23:56:34,2020-01-01 00:22:08,90,Millennium Park,90,Millennium Park,Subscriber,Chicago
3818001,2019-12-31 23:57:05,2020-01-01 00:05:46,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Chicago
3818002,2019-12-31 23:57:11,2020-01-01 00:05:45,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Chicago


### Transformación 6: creación columnas latitud y longitud para cada estación

Se quiere cambiar las columnas de station_id por su corresponidente latitud y longitud. En lugar de from_station_id, construir dos columnas una de start_lat y otra de start_long y realizar el mismo proceso para to_station_id. Esta información se consigue gracias a la tabla relacional de posiciones geográficas de columnas (stations)

Verificación para comprobar si todos los identificadores de 2019 aparecen en la tabla relacional de 2020. Si falta alguno se introducirá la posición geográfica en la tabal relacional a mano.

In [10]:
import pandas as pd

# Carga los archivos necesarios
df_trips = pd.read_csv('Chicago_2019.csv')  # Tabla de viajes
df_stations = stations  # Tabla relacional con información de ID y posición geográfica

# Verificar IDs faltantes en 'from_station_id' y obtener nombres únicos de las estaciones de inicio
id_faltantes_from = set(df_trips['from_station_id']) - set(df_stations['id_estacion'])
nombres_faltantes_from = df_trips[df_trips['from_station_id'].isin(id_faltantes_from)][['from_station_id', 'from_station_name']].drop_duplicates()

# Verificar IDs faltantes en 'to_station_id' y obtener nombres únicos de las estaciones de destino
id_faltantes_to = set(df_trips['to_station_id']) - set(df_stations['id_estacion'])
nombres_faltantes_to = df_trips[df_trips['to_station_id'].isin(id_faltantes_to)][['to_station_id', 'to_station_name']].drop_duplicates()

# Mostrar resultados
print("Nombres de estaciones de inicio faltantes:")
display(nombres_faltantes_from)

print("\nNombres de estaciones de destino faltantes:")
display(nombres_faltantes_to)

Nombres de estaciones de inicio faltantes:


,from_station_id,from_station_name
144814,360,DIVVY Map Frame B/C Station
577438,361,DIVVY CASSETTE REPAIR MOBILE STATION
848301,1,Special Events
1242368,669,LBS - BBB La Magie



Nombres de estaciones de destino faltantes:


,to_station_id,to_station_name
13246,360,DIVVY Map Frame B/C Station
198840,361,DIVVY CASSETTE REPAIR MOBILE STATION
825078,363,TS ~ DIVVY PARTS TESTING
848301,1,Special Events
1242368,669,LBS - BBB La Magie


Se puede comprobar que las estaciones faltantes en la base de datos de 2019 son simplemente estaciones de que sirven de apoyo a divvy. Por lo tanto, para nuestro estudio no es necesario incluirlas. Haremos el merge de las estaciones que hay.

In [11]:
# Hacer merge para obtener coordenadas de inicio
df_trips = df_trips.merge(df_stations, left_on='from_station_id', right_on='id_estacion', how='left') #union izqda para conservar toda la info de trayectos
df_trips.rename(columns={'latitud': 'start_lat', 'longitud': 'start_long'}, inplace=True)

# Hacer merge para obtener coordenadas de destino
df_trips = df_trips.merge(df_stations, left_on='to_station_id', right_on='id_estacion', how='left') #union izqda para conservar toda la info de trayectos
df_trips.rename(columns={'latitud': 'end_lat', 'longitud': 'end_long'}, inplace=True)

# Eliminar columnas redundantes que ya no seran necesarias
df_trips.drop(columns=['id_estacion_x', 'id_estacion_y'], inplace=True)
df_trips.drop(columns=['from_station_id', 'to_station_id'], inplace=True)

# Reordenar e imprimir resultados
df_trips = df_trips[['start_time', 'end_time', 'start_lat', 'start_long', 'end_lat', 'end_long', 'usertype', 'city']]
print("Merge realizado: ")
df_trips

Merge realizado: 


,start_time,end_time,start_lat,start_long,end_lat,end_long,usertype,city
0,2019-01-01 00:04:37,2019-01-01 00:11:07,41.8915,-87.6268,41.8916,-87.6484,Subscriber,Chicago
1,2019-01-01 00:08:13,2019-01-01 00:15:34,41.8847,-87.6277,41.8763,-87.6292,Subscriber,Chicago
2,2019-01-01 00:13:23,2019-01-01 00:27:12,41.8582,-87.6565,41.8686,-87.6862,Subscriber,Chicago
3,2019-01-01 00:13:45,2019-01-01 00:43:28,41.9227,-87.6972,41.9030,-87.6313,Subscriber,Chicago
4,2019-01-01 00:14:52,2019-01-01 00:20:56,41.8969,-87.6217,41.8923,-87.6120,Subscriber,Chicago
...,...,...,...,...,...,...,...,...
3817999,2019-12-31 23:56:13,2020-01-01 00:15:45,41.8837,-87.6487,41.9199,-87.6488,Subscriber,Chicago
3818000,2019-12-31 23:56:34,2020-01-01 00:22:08,41.8810,-87.6241,41.8810,-87.6241,Subscriber,Chicago
3818001,2019-12-31 23:57:05,2020-01-01 00:05:46,41.8728,-87.6240,41.8860,-87.6241,Subscriber,Chicago
3818002,2019-12-31 23:57:11,2020-01-01 00:05:45,41.8728,-87.6240,41.8860,-87.6241,Subscriber,Chicago


In [12]:

df_trips.to_csv('Chicago_2019.csv', index=False)
print(f"El archivo Chicago 2019 tiene {df.shape[0]} filas y {df.shape[1]} columnas")

El archivo Chicago 2019 tiene 3818004 filas y 8 columnas
